<a href="https://colab.research.google.com/github/Demshin82/AI-architect/blob/master/%D0%9F%D1%80%D0%BE%D0%BC%D0%B5%D0%B6%D1%83%D1%82%D0%BE%D1%87%D0%BD%D0%B0%D1%8F_%D0%B0%D1%82%D1%82%D0%B5%D1%81%D1%82%D0%B0%D1%86%D0%B8%D1%8F_%E2%84%963.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Установим все нужное


In [1]:
!pip install transformers[torch] datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.1 MB/s eta 0:00:00


Импорт датасета и метрики оценки качества. Она позволит произвести валидацию, так как есть оценка веростных ответов для конкретного датасета.


In [2]:
import numpy as np
import collections
import evaluate
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader, Dataset
from transformers import DistilBertForQuestionAnswering
from transformers import AutoTokenizer
from transformers import default_data_collator
from transformers import Trainer, TrainingArguments
from transformers import pipeline
from tqdm.auto import tqdm

metric = evaluate.load("squad")
dataset = load_dataset("squad")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Так как у нас не так много памяти возьмем 5000 сэмплов для обучения и 500 для валидации

In [3]:
dataset['train'] = dataset['train'].select( range(5000))
dataset['validation'] = dataset['validation'].select(range(500))

Определим модель и токенизер. Будем файн-тюнить модельку DistilBertForQuestionAnswering

In [4]:
checkpoint = "distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)
model = DistilBertForQuestionAnswering.from_pretrained(checkpoint).to(device)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

cuda


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
def QA_answer(model_checkpoint, tokenizer, context, question):
  """
  Функция для ответа на вопросы
  """
  question_answerer = pipeline("question-answering", model=model_checkpoint)

  result = question_answerer(question=question, context=context)
  print(f"Question: {question} \n Answer: {result['answer']}")

In [6]:
i = np.random.randint(len(dataset['validation']))
context = dataset['validation'][i]['context']
question = dataset['validation'][i]['question']
answer = dataset['validation'][i]['answers']['text']
print(f"Q: {question} \n C: {context} \n A: {answer}")
print("---"*30)
print("Not fine-tuned model")
QA_answer(checkpoint, tokenizer, context, question)

Q: What color were the Bronco's uniforms in Super Bowl 50? 
 C: As the designated home team in the annual rotation between AFC and NFC teams, the Broncos elected to wear their road white jerseys with matching white pants. Elway stated, "We've had Super Bowl success in our white uniforms." The Broncos last wore matching white jerseys and pants in the Super Bowl in Super Bowl XXXIII, Elway's last game as Denver QB, when they defeated the Atlanta Falcons 34–19. In their only other Super Bowl win in Super Bowl XXXII, Denver wore blue jerseys, which was their primary color at the time. They also lost Super Bowl XXI when they wore white jerseys, but they are 0-4 in Super Bowls when wearing orange jerseys, losing in Super Bowl XII, XXII, XXIV, and XLVIII. The only other AFC champion team to have worn white as the designated home team in the Super Bowl was the Pittsburgh Steelers; they defeated the Seattle Seahawks 21–10 in Super Bowl XL 10 seasons prior. The Broncos' decision to wear white me

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Question: What color were the Bronco's uniforms in Super Bowl 50? 
 Answer: : black jerseys with


Далее будут функции препроцессинга данных для валидации и обучения.

In [7]:
def preprocess_training_examples(examples, max_length = 512, stride = 128):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [8]:
def preprocess_validation_examples(examples, max_length = 512, stride = 128):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [9]:
def compute_metrics(start_logits, end_logits, features, examples,
                    n_best = 20, max_answer_length = 30):

    example_to_features = collections.defaultdict(list)

    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

Готовим датасеты

In [10]:
train_dataset = dataset["train"].map(
    preprocess_training_examples,
    batched=True,
    remove_columns=dataset["train"].column_names,
)

validation_dataset = dataset["validation"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=dataset["validation"].column_names,
)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Обучения модели будем выполнять через доступную функцию Trainer

In [11]:
args = TrainingArguments(
    output_dir='./fine_tuned_model',
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    save_steps=100,
    logging_steps=200
)


trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer
)
trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
200,3.684600
400,2.102700
600,1.569500
800,1.266600


TrainOutput(global_step=942, training_loss=2.013216832640824, metrics={'train_runtime': 731.9544, 'train_samples_per_second': 20.53, 'train_steps_per_second': 1.287, 'total_flos': 1963324134180864.0, 'train_loss': 2.013216832640824, 'epoch': 3.0})

In [12]:
predictions, _, _ = trainer.predict(validation_dataset)
start_logits, end_logits = predictions
compute_metrics(start_logits, end_logits, validation_dataset, dataset["validation"])

  0%|          | 0/500 [00:00<?, ?it/s]

{'exact_match': 60.0, 'f1': 67.31372385099834}

метрики слабоваты, но уже ничего. Попробуем на нашем вопросе. Для этого возьмем последний чекпоинт из тренированного датасета: в моем случае это 942 итерация

In [13]:
QA_answer('/content/fine_tuned_model/checkpoint-942', trainer.tokenizer, context, question)

Question: What color were the Bronco's uniforms in Super Bowl 50? 
 Answer: blue jerseys


Сразу видно, что ответ на вопрос уже улучшился.

Проверим модельку на специфических вопросах о геологии

In [14]:
q = "What is the term used to describe the process of breaking down rocks into smaller particles through physical or chemical means?"
c = "Weathering is a fundamental concept in geology."
QA_answer('/content/fine_tuned_model/checkpoint-942', trainer.tokenizer, c, q)

Question: What is the term used to describe the process of breaking down rocks into smaller particles through physical or chemical means? 
 Answer: Weathering


Если ответ есть в контексте, то модель справляется хорошо. Проверим что будет, если ответ частично или полностью отсутствует в контексте.

In [15]:
q = "What is the process by which one type of rock transforms into another type of rock under intense heat and pressure?"
c = "Geological processes can cause rocks to change from one type to another."
QA_answer('/content/fine_tuned_model/checkpoint-942', trainer.tokenizer, c, q)

Question: What is the process by which one type of rock transforms into another type of rock under intense heat and pressure? 
 Answer: Geological


Ожидаемый ответ - Metamorphic rock. Попробуем более простые, но рукописные вопросы\ответы

In [16]:
q = "How many countries in the world?"
c = "Countries of the world according to UN – 195. A place with its own borders and fully independent government. This is the definition of a sovereign state or a country by the United Nations (UN). According to the UN, there are 193 countries in the world and 2 observer states (Palestine and Vatican City). So in total, there are 195 countries in the world."
QA_answer('/content/fine_tuned_model/checkpoint-942', trainer.tokenizer, c, q)

Question: How many countries in the world? 
 Answer: 193


In [17]:
q = "whose windows were washed?"
c = "While we were walking, we were watching window washers wash Washington’s windows with warm washing water."
QA_answer('/content/fine_tuned_model/checkpoint-942', trainer.tokenizer, c, q)

Question: whose windows were washed? 
 Answer: Washington’s windows with warm washing water


Со сложной скороговоркой модель справилась частично, но думаю, что полный датасет решит эту проблму.